# Artificial Art with more then one style image

In diesem Projekt wollen wir uns mit dem Erstellen von Bildern aus Content- und Style- Bildern mithilfe von maschinellem Lernen beschäftigen. 

Als Grundlage dieser Arbeit dient [dieses Notebook von Harish Narayanan](https://github.com/hnarayanan/artistic-style-transfer/blob/master/notebooks/6_Artistic_style_transfer_with_a_repurposed_VGG_Net_16.ipynb).
Wir erweitern dieses um unsere eigenen Gedanken, und führen die Option ein, mehrere Content- und Style- Bilder zu verwenden.

Als Frameworks nutzen wir `keras`, welches Tensorflow (1.0) als Backend benutzt, sowie `scypy` für einen Optimierungsalgorithmus sowie `PIL`, um Bilder laden und bearbeiten zu können.

In [28]:
import numpy as np
import copy

import time
from PIL import Image

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b

Zunächst geben wir die Größe der Bilder an. Egal, welche Größe die Ausgangsbilder haben, sie werden alle auf diese Größe skaliert.

Um maschinelles Lernen betreiben zu können, werden die Bilder in Numpy-Arrays konvertiert. Es wird eine Dimension hinzugefügt, damit am Ende alle Bilder in einem Datenobjekt zusammengefasst werden können.

Wir werden das viel verwendete VGG16-Modell benutzen. Da dieses Modell ein spezielles Datenformat verlangt, müssen die Daten noch folgendermaßen angepasst werden:
* Von allen Pixeln werden die mittleren RGB-Werte in den entsprechenden Farbchannels abgezogen.
* Die Farbchannels werden in ihrer Reihenfolge umgekehrt.

In [29]:
height = 512
width = 512

content_image_paths = ['images/hulk.jpg', "images/giger.jpg"]
content_imgs = []
for content_img_path in content_image_paths:
    content_img = Image.open(content_img_path)
    content_img = content_img.resize((height, width))
    content_arr = np.asarray(content_img, dtype='float32')
    content_arr = np.expand_dims(content_arr, axis=0)
    content_arr[:, :, :, 0] -= 103.939
    content_arr[:, :, :, 1] -= 116.779
    content_arr[:, :, :, 2] -= 123.68
    # Convert from RGB to BGR
    content_arr = content_arr[:, :, :, ::-1]
    content_img = backend.variable(content_arr)
    content_imgs.append(content_img)

print(content_imgs)

[<tf.Variable 'Variable_12:0' shape=(1, 512, 512, 3) dtype=float32_ref>, <tf.Variable 'Variable_13:0' shape=(1, 512, 512, 3) dtype=float32_ref>]


In [30]:
style_image_paths = ['images/style1.jpg', 'images/style2.jpg']
style_imgs = []
for style_img_path in style_image_paths:
    style_img = Image.open(style_img_path)
    style_img = style_img.resize((height, width))
    style_arr = np.asarray(style_img, dtype='float32')
    style_arr = np.expand_dims(style_arr, axis=0)
    style_arr[:, :, :, 0] -= 103.939
    style_arr[:, :, :, 1] -= 116.779
    style_arr[:, :, :, 2] -= 123.68
    # Convert from RGB to BGR
    style_arr = style_arr[:, :, :, ::-1]
    style_img = backend.variable(style_arr)
    style_imgs.append(style_img)


In unserem Falle sind wir nicht daran interessiert, das Klassifikationsnetzwerk zur tatsächlichen Klassifikation zu benutzen, sondern wollen statt Weights und Biases anzupassen unsere Inputs anpassen, um die Kostenfunktion zu minimieren. Dafür brauchen wir in den Inputs neben den Ausgangsbildern noch Platz für das Ergebnisbild. Dafür erstellen wir einen Placeholder, der dieselben Dimensionen hat.

In [31]:
# Channels as the last dimension, using backend Tensorflow
combination_img = backend.placeholder((1, height, width, 3))

Wir konkatenieren alle Ausgangsbilder und den Platzhalter für das Ergebnisbild in einen Tensor, der dem Modell als Input dient.

In [32]:
# We now finally have the content image variable, style image variables and combination image placeholder
# that we will concatenate to build a final input tensor to build our computation graph on top of, essentially
# we pass all these inputs to the network as if they are a part of a batch so they are all run parallely and we
# use the features generated to modify our combination image based on the losses. TODO: Delete this comment

all_imgs = []
for content_img in content_imgs:
    all_imgs.append(content_img)
for style_img in style_imgs:
    all_imgs.append(style_img)
all_imgs.append(combination_img)
input_tensor = backend.concatenate(all_imgs, axis=0)

Nun wird das Netzwerk initialisiert. Genaueres zum Aufbau in [diesem Notebook](VGG16_mnist.ipynb). Wir nutzen das bereits mit Daten von Imagenet trainierte Netzwerk und lassen die Fully-Connected-Layers weg, da diese nur für die Klassifikation benötigt werden und keine für uns relevanten Informationen enthalten.

In [33]:
model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [34]:
layers = dict([(layer.name, layer.output) for layer in model.layers])
layers

{'block1_conv1': <tf.Tensor 'block1_conv1_2/Relu:0' shape=(5, 512, 512, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2_2/Relu:0' shape=(5, 512, 512, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool_2/MaxPool:0' shape=(5, 256, 256, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1_2/Relu:0' shape=(5, 256, 256, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2_2/Relu:0' shape=(5, 256, 256, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool_2/MaxPool:0' shape=(5, 128, 128, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1_2/Relu:0' shape=(5, 128, 128, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2_2/Relu:0' shape=(5, 128, 128, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3_2/Relu:0' shape=(5, 128, 128, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool_2/MaxPool:0' shape=(5, 64, 64, 256) dtype=float32>,
 'block4_conv1': <tf.Tensor 'block4_conv1_2/Relu:0' shape=(5, 64, 64, 51

Wie man sehen kann, schlägt sich die Bildgröße in allen Schichten in der Parameteranzahl wider. Dies bedeutet, dass eine Erhöhung der Auflösung zu einer deutlich überproportional längeren Berechnungsdauer führt.

Um ein möglichst gutes Endergebnis zu erhalten, muss dafür gesorgt werden, dass sowohl der Inhalt der Content-Bilder erhalten bleibt, als auch der Style der Style-Bilder angemessen im Ergebnisbild zu erkennen ist. Da diese mit einander in Konflikt stehen, nutzen wir als Loss eine gewichtete Summe. 

Im Folgenden werden die einzelnen Bestandteile genauer erläutert. 

Die Werte der Gewichte haben wir aus [Narayanans Notebook](https://github.com/hnarayanan/artistic-style-transfer/blob/master/notebooks/6_Artistic_style_transfer_with_a_repurposed_VGG_Net_16.ipynb) übernommen. Durch viel Ausprobieren fand er diese Werte, die zu einem subjektiv als gut empfundenen Ergebnis geführt haben. Es bedeutet nicht, dass der Style 200x wichtiger als der Content ist, da die Ergebnisse der einzelnen Loss-Funktionen nicht normalisiert werden und somit nicht direkt miteinander vergleichbar sind.

In [35]:
content_weight = 0.025
style_weight = 5.0
tv_weight = 1.0

In [36]:
loss = backend.variable(0.)

Der Content-Loss ist die euklidische Distanz zwischen den Outputs einer ausgewählten Schicht für den Content und unser Ergebnisbild. 

TODO: verschiedene Schichten ausprobieren und Ergebnis beschreiben.

Die höheren Schichten erkennen eher größere Strukturen und weniger konkrete Details.

In [37]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

# Add content loss to this layer
layer_features = layers['block2_conv2']
content_features = layer_features[:len(content_imgs) - 1, :, :, :]
print(content_imgs)
print(style_imgs)
combination_features = layer_features[len(content_imgs) + len(style_imgs), :, :, :]

loss += content_weight * content_loss(content_features, combination_features)

[<tf.Variable 'Variable_12:0' shape=(1, 512, 512, 3) dtype=float32_ref>, <tf.Variable 'Variable_13:0' shape=(1, 512, 512, 3) dtype=float32_ref>]
[<tf.Variable 'Variable_14:0' shape=(1, 512, 512, 3) dtype=float32_ref>, <tf.Variable 'Variable_15:0' shape=(1, 512, 512, 3) dtype=float32_ref>]


Der Style-Loss ist der interessanteste Teil dieses Algorithmus. Hier wird eine sogenannte Gram-Matrix zwischengeschaltet, die die Korrelation der Outputs der Neuronen verschiedener Schichten für alle Style-Bilder repräsentiert. Somit werden Informationen über die einzelnen Outputs verloren, aber Informationen über die Zusammenhänge der Neuronen untereinander gewonnen. Dies ist ein erstaunlich guter Indikator für Stil-Ähnlichkeit.

In der Gram Matrix werden alle Outputs der betroffenen Neuronen paarweise miteinander multipliziert. Dafür wird zunächst ein Spaltenvektor aus allen n Outputs generiert und dieser mit seiner Transposition multipliziert, sodass eine n x n - Matrix entsteht.

In [38]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

feature_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3'] # TODO: try out different layers, compare results and write about them
for layer_name in feature_layers:
    layer_features = layers[layer_name]
    for style_img_idx in range(len(style_imgs)):
        style_features = layer_features[1 + style_img_idx, :, :, :]
        combination_features = layer_features[1 + len(style_imgs), :, :, :]
        style_l = style_loss(style_features, combination_features)
        loss += (style_weight / (len(feature_layers)*len(style_imgs))) * style_l

TODO: try out with and without total variation loss.

Damit das Ergebnisbild nicht zu verrauscht ist, wird noch ein dritter Term in die Loss-Funktion aufgenommen: der Total Variation Loss. Hierbei wird einmal das Bild ohne letzte Zeile über das Bild ohne erste Zeile gelegt und die euklidische Distanz berechnet. Für die Spalten wird analog vorgegangen und das Ergebnis aufaddiert. So wird insgesamt jeder Pixel einmal mit jedem direkten Nachbarn (oben, unten, rechts, links) verglichen und zu große Unterschiede werden bestraft. Somit wird einem zu großen Rauschen entgegengewirkt.

In [39]:
# Total variation loss to ensure the image is smooth and continuous throughout

def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

loss += tv_weight * total_variation_loss(combination_img)

In [40]:
# Since all of these variables are nodes in our computational graph, we can directly
# calculate the gradients
grads = backend.gradients(loss, combination_img)

In [41]:
outputs = [loss]
outputs += grads
# Create the function from input combination_img to the loss and gradients
f_outputs = backend.function([combination_img], outputs)

In [42]:
# We finally have the gradients and losses at the combination_img computed as variables
# and we can use any standard optimization function to optimize combination_img

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.0

def transform_back(x):
    x1 = copy.deepcopy(x)
    x1 = x1.reshape((height, width, 3))
    # Convert back from BGR to RGB to display the image
    x1 = x1[:, :, ::-1]
    x1[:, :, 0] += 103.939
    x1[:, :, 1] += 116.779
    x1[:, :, 2] += 123.68
    x1 = np.clip(x1, 0, 255).astype('uint8')
    return Image.fromarray(x1)

transform_back(x).save('before.bmp')

iters = 2

for i in range(iters):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

    transform_back(x).save('result' + str(i) + '.bmp')

('Start of iteration', 0)
